In [ ]:
import pygame
import sys
import numpy as np # Used for board initialization with zeros, good practice
import copy # Needed for deep copies of the board in minima

pygame.init()

# Global Constants and Game State Variables ---
# Screen dimensions and drawing parameters
WIDTH = 300
HEIGHT = 300
BOARD_ROWS = 3 # Corrected typo from BOARD_ROW
BOARD_COLS = 3
SQUARE_SIZE = WIDTH // BOARD_COLS # Recalculate based on corrected WIDTH and BOARD_COLS

LINE_WIDTH = 5
CIRCLE_RADIUS = SQUARE_SIZE // 3
CIRCLE_WIDTH = 15
CROSS_WIDTH = 25
SPACE = SQUARE_SIZE // 4 # For drawing the cross/X

# Colors (RGB tuples)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (180, 180, 180)
BLUE_WIN_LINE = (0, 0, 200) # A distinct color for winning lines

# Pygame Display Surface
Screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Tic Tac Toe')
Screen.fill(BLACK) # Fill the screen initially

# Game board (using numpy for easy initialization, then convert to list)
board = np.zeros((BOARD_ROWS, BOARD_COLS), dtype=int).tolist()

# Game state variables
player = 1 # 1 for human, 2 for AI/computer
game_over = False
current_winner_player = None # Store the winning player (1 or 2)

#Drawing Functions

def draw_lines(color=WHITE): #Draws the tic-tac-toe grid lines on the Screen.
    
    # Horizontal lines
    for i in range(1, BOARD_ROWS):
        pygame.draw.line(Screen, color, (0, SQUARE_SIZE * i), (WIDTH, SQUARE_SIZE * i), LINE_WIDTH)
    # Vertical lines
    for i in range(1, BOARD_COLS):
        pygame.draw.line(Screen, color, (SQUARE_SIZE * i, 0), (SQUARE_SIZE * i, HEIGHT), LINE_WIDTH)

def draw_figures(figure_color=WHITE): # Added figure_color parameter to fix 'color' NameError
    #Draws X's and O's on the board based on the board state.
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 1: # Player 1 (Circles)
                pygame.draw.circle(Screen, figure_color, # Use passed figure_color
                                   (int(col * SQUARE_SIZE + SQUARE_SIZE // 2),
                                    int(row * SQUARE_SIZE + SQUARE_SIZE // 2)),
                                   CIRCLE_RADIUS, CIRCLE_WIDTH)
            elif board[row][col] == 2: # Player 2 (Crosses/X's)
                # First line of the cross (top-left to bottom-right)
                pygame.draw.line(Screen, figure_color, # Use passed figure_color, fixed 'screen' to 'Screen'
                                 (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SPACE),
                                 (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE),
                                 CROSS_WIDTH)
                # Second line of the cross (top-right to bottom-left)
                pygame.draw.line(Screen, figure_color, # Use passed figure_color, fixed 'screen' to 'Screen'
                                 (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE),
                                 (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SPACE),
                                 CROSS_WIDTH)

# Winning Line Drawing Helper Functions
def draw_vertical_winning_line(col, color):
    #Draws a vertical winning line.
    posX = col * SQUARE_SIZE + SQUARE_SIZE // 2
    pygame.draw.line(Screen, color, (posX, LINE_WIDTH), (posX, HEIGHT - LINE_WIDTH), LINE_WIDTH)

def draw_horizontal_winning_line(row, color):
    #Draws a horizontal winning line.
    posY = row * SQUARE_SIZE + SQUARE_SIZE // 2
    pygame.draw.line(Screen, color, (LINE_WIDTH, posY), (WIDTH - LINE_WIDTH, posY), LINE_WIDTH)

def draw_asc_diagonal_winning_line(color):
    #Draws an ascending diagonal winning line (bottom-left to top-right).
    pygame.draw.line(Screen, color, (LINE_WIDTH, HEIGHT - LINE_WIDTH), (WIDTH - LINE_WIDTH, LINE_WIDTH), LINE_WIDTH)

def draw_desc_diagonal_winning_line(color):
    #Draws a descending diagonal winning line (top-left to bottom-right)."""
    pygame.draw.line(Screen, color, (LINE_WIDTH, LINE_WIDTH), (WIDTH - LINE_WIDTH, HEIGHT - LINE_WIDTH), LINE_WIDTH)

    
    
    
#Game Logic Functions 

def mark_square(row, col, player_num): # Renamed 'player' to 'player_num' to avoid confusion with global 'player'
    #Marks a square on the board with the given player's number.
    global board
    board[row][col] = player_num

    
def available_square(row, col):
    #Checks if a square is empty (0) and within valid board bounds.
    if 0 <= row < BOARD_ROWS and 0 <= col < BOARD_COLS:
        return board[row][col] == 0
    return False # Out of bounds is not available


def is_board_full(check_board=None): # can give default as board itself
    #Checks if the board is full.
    if check_board is None: # Use global board if no specific board is provided
        check_board = board
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if check_board[row][col] == 0:
                return False
    return True # If no empty squares found, board is full



def check_win(player_num, check_board=None): 
    #Checks if the given player has won the game
    if check_board is None: # Use global board if no specific board is provided
        check_board = board

    # Check vertical wins
    for col in range(BOARD_COLS):
        if check_board[0][col] == player_num and check_board[1][col] == player_num and check_board[2][col] == player_num:
            return True

    # Check horizontal wins
    for row in range(BOARD_ROWS):
        if check_board[row][0] == player_num and check_board[row][1] == player_num and check_board[row][2] == player_num:
            return True

    # Check descending diagonal win (top-left to bottom-right)
    if check_board[0][0] == player_num and check_board[1][1] == player_num and check_board[2][2] == player_num:
        return True

    # Check ascending diagonal win (bottom-left to top-right)
    if check_board[2][0] == player_num and check_board[1][1] == player_num and check_board[0][2] == player_num:
        return True

    return False # No win found




# Minimax AI Functions 

def minimax(minimax_board, depth, is_maximizing):
    """
    Minimax algorithm to determine the best move for the AI.
    Player 1 is human (minimizing), Player 2 is computer (maximizing).
    """
    if check_win(2, minimax_board): # Computer wins
        return float('inf')
    elif check_win(1, minimax_board): # Human wins
        return float('-inf')
    elif is_board_full(minimax_board): # Tie
        return 0

    if is_maximizing:
        best_score = float('-inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if minimax_board[row][col] == 0:
                    minimax_board[row][col] = 2 # Simulate AI's move
                    score = minimax(minimax_board, depth + 1, False) # Recurse for human's turn
                    minimax_board[row][col] = 0 # Undo move (backtrack)
                    best_score = max(best_score, score)
        return best_score
    else: # is_minimizing (human's turn)
        best_score = float('inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if minimax_board[row][col] == 0:
                    minimax_board[row][col] = 1 # Simulate human's move
                    score = minimax(minimax_board, depth + 1, True) # Recurse for AI's turn
                    minimax_board[row][col] = 0 # Undo move (backtrack)
                    best_score = min(best_score, score)
        return best_score

def best_move():
    """Calculates and executes the AI's best move using minimax."""
    global board
    best_score = float('-inf')
    move = (-1, -1) 
    
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 0:
                # Create a deep copy of the board for minimax to avoid modifying the actual game board
                temp_board = copy.deepcopy(board)
                temp_board[row][col] = 2 # Simulate AI's move on the copy
                score = minimax(temp_board, 0, False) # Human will try to minimize this score
                if score > best_score:
                    best_score = score
                    move = (row, col) # Update the best move

    if move != (-1, -1): # If a valid move was found
        mark_square(move[0], move[1], 2) # Execute the best move on the actual board
        return True # AI made a move
    return False # No move made 




# Game Management Functions

def restart_game():
    """Resets the game board and state."""
    global board, player, game_over, current_winner_player
    board = np.zeros((BOARD_ROWS, BOARD_COLS), dtype=int).tolist() # Reset board
    player = 2
    game_over = False
    current_winner_player = None

    Screen.fill(BLACK) # Clear screen
    draw_lines(WHITE) # Redraw initial grid
    # No need to display in this function; the main loop will handle display update

# --- Main Game Loop (for standalone Pygame application) ---
# This part is designed for a traditional Pygame window, not Jupyter.
# Running this in Jupyter will block the kernel.

draw_lines(WHITE) # Draw initial lines for the first frame

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            mouseX, mouseY = event.pos
            clicked_col = mouseX // SQUARE_SIZE
            clicked_row = mouseY // SQUARE_SIZE

            if available_square(clicked_row, clicked_col): # Check availability using clicked_row, clicked_col
                if player == 1:
                    mark_square(clicked_row, clicked_col, 1)
                    if check_win(1, board):
                        game_over = True
                        current_winner_player = 1
                    else:
                        player = 2 # Switch to AI's turn
                else: # player == 2 (AI's turn if no human move was made, but typically AI moves here)
                    # This branch is usually for player 2 (human) if it's a 2-player game.
                    # For AI vs Human, AI makes move based on best_move()
                    pass # AI's move handled outside this block

                # Redraw figures after a valid move
                Screen.fill(BLACK)
                draw_lines(WHITE)
                draw_figures(WHITE)


        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r: # 'r' key to restart
                restart_game()

    # AI's turn
    if player == 2 and not game_over:
        if best_move():
            if check_win(2, board):
                game_over = True
                current_winner_player = 2
            elif is_board_full(board): # Check for draw after AI move
                game_over = True
            else:
                player = 1 # Switch back to human
        # Redraw figures after AI move
        Screen.fill(BLACK)
        draw_lines(WHITE)
        draw_figures(WHITE)

        
        
    # Draw winning line if game is over with a winner
    if game_over and current_winner_player is not None:
        winning_color = GREEN if current_winner_player == 1 else RED
        # Redraw everything to ensure winning line is on top
        Screen.fill(BLACK)
        draw_lines(WHITE)
        draw_figures(WHITE) # Draw original figures
        # Determine and draw the specific winning line
        if check_win(current_winner_player, board):
            # Check all win conditions to draw the correct line
            if board[0][0] == current_winner_player and board[1][1] == current_winner_player and board[2][2] == current_winner_player:
                draw_desc_diagonal_winning_line(winning_color)
            elif board[2][0] == current_winner_player and board[1][1] == current_winner_player and board[0][2] == current_winner_player:
                draw_asc_diagonal_winning_line(winning_color)
            else: # Check rows and columns
                for c in range(BOARD_COLS):
                    if board[0][c] == current_winner_player and board[1][c] == current_winner_player and board[2][c] == current_winner_player:
                        draw_vertical_winning_line(c, winning_color)
                        break
                for r in range(BOARD_ROWS):
                    if board[r][0] == current_winner_player and board[r][1] == current_winner_player and board[r][2] == current_winner_player:
                        draw_horizontal_winning_line(r, winning_color)
                        break
    elif game_over and current_winner_player is None and is_board_full(board): # It's a draw
        Screen.fill(BLACK)
        draw_lines(GRAY)
        draw_figures(GRAY)


    pygame.display.update()

In [ ]:
def isWinner(board,player):
    winCombination = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  
        [0, 4, 8], [2, 4, 6]              
    ]
    return any(all(board[i] == player for i in combo)for combo in winCombination)

def isDraw(board):
    return all(cell != " " for cell in board)

def avlMoves(board):
    PossibleMoves = []
    for i,cell in enumerate(board):
        if cell == " ":
            PossibleMoves.append(i)

    return PossibleMoves

def minimax(board,isMax):
    if isWinner(board,"X"):
        return 1
    if isWinner(board,"0"):
        return -1
    else:
        return 0

    if isMax:
        MaxScore = -float('inf')
        for move in avlMoves(board):
            board[move] = "X"
            score = minimax(board,False)
            board[move] = " "
            MaxScore = max(score,MaxScore)
        return MaxScore
    else:
        MaxScore = float('inf')
        for move in avlMoves(board):
            board[move] = "O"
            score = minimax(board,True)
            board[move] = " "
            MaxScore = min(score,MaxScore)
        return MaxScore

def FindBestMove(board):
    bestScore = -float('inf')
    bestMove = None
    for move in avlMoves(board):
        board[move] = "X"
        score = minimax(board,False)
        board[move] = " "
        if score > bestScore:
            bestScore = score
            bestMove = move

    return bestMove

def print_board(board):
    print()
    for i in range(3):
        row = board[i*3:i*3+3]
        print(" | ".join(row))
        if i < 2:
            print("--+---+--")
    print()

def play_game():
    board = [" " for _ in range(9)]
    human = "X"
    ai = "O"
    
    while True:
        print_board(board)

        move = input("Your move (1-9): ")
        if not move.isdigit() or int(move) not in range(1, 10):
            print("Invalid input. Try again.")
            continue
        move = int(move) - 1
        if board[move] != " ":
            print("Cell already taken. Try again.")
            continue
        board[move] = human

        if isWinner(board, human):
            print_board(board)
            print("🎉 You win!")
            break
        if isDraw(board):
            print_board(board)
            print("It's a draw!") 
            break

        print("AI is thinking...")
        ai_move = FindBestMove(board)
        board[ai_move] = ai

        if isWinner(board,ai):
            print_board(board)
            print("💻 AI wins!")
            break
        if isDraw(board):
            print_board(board)
            print("It's a draw!")
            break

play_game()